---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [10]:
import pandas as pd
import numpy as np
import re
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [2]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [30]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    df_university_towns = pd.read_table('university_towns.txt', header=None)
    state = []
    list_towns_by_state = []
    towns_by_state = []
    for town in df_university_towns.iloc[0].values:
        if '[edit]' in town:
            state.append(town[:-6])
        else:
            towns_by_state.append(re.sub('\d+','',re.sub('\]','',re.sub('\[','',town))))

    for index in range(1,len(df_university_towns)):
        item = df_university_towns.iloc[index].values[0]
        if '[edit]' in item:
            state.append(item[:-6])
            list_towns_by_state.append(towns_by_state)
            towns_by_state = []
        else:
            towns_by_state.append(re.sub('\d+','',re.sub('\]','',re.sub('\[','',item[0:item.find('(')-1]))))

    state_and_town_final_list = []

    for s, t_l in zip(state,list_towns_by_state):
        for t in t_l:
            state_and_town_final_list.append([s,t])

    df_uni_towns = pd.DataFrame(state_and_town_final_list, columns=['State','RegionName'])
    return df_uni_towns


In [31]:
def GDP_data():
    df_GDP = pd.read_excel('gdplev.xls')
    df_GDP = (df_GDP.drop(['Current-Dollar and "Real" Gross Domestic Product','Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 5'],axis=1)
              .rename(columns={'Unnamed: 4': 'Quarter', 'Unnamed: 6': 'GDP ($ Billion)'}))[7:]
    df_GDP = df_GDP.T[0:2].T
    return df_GDP.set_index('Quarter')

def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    df_GDP = GDP_data()
    df_GDP = df_GDP[(df_GDP.index.get_loc('2000q1')):]
    for i in range(1,len(df_GDP)):
        if (df_GDP.iloc[i]['GDP ($ Billion)'] < df_GDP.iloc[i - 1]['GDP ($ Billion)']) and (df_GDP.iloc[i]['GDP ($ Billion)'] > df_GDP.iloc[i + 1]['GDP ($ Billion)']):
            recession_start = df_GDP.iloc[i].name
            return recession_start
    return None

In [32]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    recession_start = get_recession_start()
    df_GDP = GDP_data()
    index = df_GDP.index.get_loc(recession_start)
    for i in range(index + 2, len(df_GDP)):
        if (df_GDP.iloc[i]['GDP ($ Billion)'] > df_GDP.iloc[i - 1]['GDP ($ Billion)']) and (df_GDP.iloc[i - 1]['GDP ($ Billion)'] > df_GDP.iloc[i - 2]['GDP ($ Billion)']):
            return df_GDP.iloc[i].name
    return None

In [33]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    min_GDP_index = 0
    df_recession_quarters = GDP_data()[GDP_data().index.get_loc(get_recession_start()) :GDP_data().index.get_loc(get_recession_end()) + 1]
    min_GDP = df_recession_quarters.iloc[0]['GDP ($ Billion)']
    for i in range(0,len(df_recession_quarters.index)):
        if min_GDP > df_recession_quarters.iloc[i]['GDP ($ Billion)']:
            min_GDP = df_recession_quarters.iloc[i]['GDP ($ Billion)']
            min_GDP_index = i
    return df_recession_quarters.iloc[min_GDP_index].name

In [34]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    df_housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    df_housing_data = df_housing_data.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis=1)
    df_housing_data = df_housing_data.replace({'State': states})
    df_housing_data = df_housing_data.set_index(['State', 'RegionName'])
    df_housing_data = df_housing_data.replace(to_replace='NaN', value=np.NaN)
    df_housing_data = df_housing_data.convert_objects(convert_numeric=True).sort()
    index = list(df_housing_data.columns.values).index('2000-01')
    df_housing_data = df_housing_data.T[index:].T
    columns_by_quarter = []
    for month_year in df_housing_data.columns:
        year = month_year[0:4]
        month = int(month_year[-2:])
        if month in range(1,4):
            quarter = 'q1'
        if month in range(4,7):
            quarter = 'q2'
        if month in range(7,10):
            quarter = 'q3'
        if month in range(10,13):
            quarter = 'q4'
        columns_by_quarter.append(year + quarter)

    df_housing_data.columns = columns_by_quarter
    index_col = 0

    while index_col < len(columns_by_quarter):
        df_housing_data[columns_by_quarter[index_col]] = df_housing_data.T[index_col : index_col + 3].mean(axis=0)
        index_col+=3

    df_housing_data = df_housing_data.loc[:,~df_housing_data.columns.duplicated()]
    return df_housing_data

In [47]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    housing_data = convert_housing_data_to_quarters()
    recession_start = get_recession_start()
    recession_end = get_recession_end()
    recession_botton = get_recession_bottom()
    recession_start_index = housing_data.columns.get_loc(recession_start)
    recession_end_index = housing_data.columns.get_loc(recession_end)
    town_data = get_list_of_university_towns()
    town_data = town_data.set_index(['State', 'RegionName'])
    #housing_data['Ratio'] = housing_data.iloc[:, start_test - 1] / housing_data.iloc[:, end_test]
    #uni_town_prices = pd.merge(town_list, housing_data)
    housing_data['Price Ratio'] = (housing_data[recession_start] - housing_data[recession_end])/housing_data[recession_botton]
    merge_uni_towns = pd.merge(town_data, housing_data, how='inner', right_index=True, left_index=True)
    merge_uni_towns['University Town'] = True
    merge_non_uni_towns = pd.merge(merge_uni_towns, housing_data, how='outer', right_index=True, left_index=True)
    merge_non_uni_towns['University Town'] = False
    t,p = ttest_ind(merge_non_uni_towns['Price Ratio_y'].dropna(),merge_uni_towns['Price Ratio'].dropna())
    different = True if p < 0.01 else False
    better = "university town" if merge_uni_towns['Price Ratio'].mean() < merge_non_uni_towns['Price Ratio_y'].mean() else "non-university town"
    return (different, p, better)


In [48]:
run_ttest()

(True, 0.0024006826615501822, 'university town')